# Librerias de apoyo

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv

from glob import glob
from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import imutils

In [2]:
import math
def area_elipse(a,b):
    
    area=(a/2)*(b/2)*math.pi
    return area

def perimetro_elipse(a,b):
    #usando aproximación de Ramanujan
    perimetro=math.pi*(3.0*(a+b)-math.sqrt((3.0*a+b)*(a+3.0*b)))
    return perimetro

def hsv_convert(image):
    return cv.cvtColor(img, cv.COLOR_BGR2HSV)

# Regresa la imagen en escala de grises
def grey(image):
    return cv.cvtColor(image, cv.COLOR_BGR2GRAY)

#Definimos una función que retorne la mascara para el color rojo
def f_mask_green(image):
#    lower_green = np.array([6,100,7])
    lower_green = np.array([6,64,7])
    upper_green = np.array([78,255,255])
    
    img_hsv=hsv_convert(image)
    mask_green = cv.inRange(img_hsv,lower_green,upper_green)
    
    return mask_green


In [3]:
def contornos(img):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    blurred = cv.medianBlur(gray,5)#13)
    thr, imagen_th_grey = cv.threshold(blurred, 1,255, cv.THRESH_BINARY)
    edged = cv.Canny(imagen_th_grey, 1, 150)

    cnts = cv.findContours(edged.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    (cnts, _) = contours.sort_contours(cnts)
    

    mayor=0
    mayor_i=0
    #print(len(cnts[0])) 
    #print(len(cnts[1])) 
    
    for i in range(len(cnts)) :
        if len(cnts[i]) > mayor :
            mayor_i = i
            mayor = len(cnts[i])
    return [cnts[mayor_i]]

In [4]:
# función para quitar el pendúnculo
def digit(img):
    kernel = np.ones((5,5),np.uint8)
    img_mask=f_mask_green(img)
    img_mask = cv.medianBlur(img_mask,13)
    img_mask=np.logical_not(img_mask)
    img[img_mask] = 0
    img_opening = cv.morphologyEx(img, cv.MORPH_OPEN, kernel, iterations=9) #12
    #img_opening = cv.dilate(img_opening,kernel,iterations = 10) #
    img_grey=grey(img_opening)
    thr, imagen_th_grey = cv.threshold(img_grey, 4,255, cv.THRESH_TOZERO)
    img_mask2=np.logical_not(imagen_th_grey)
    img[img_mask2] = 0
    
    cnts= contornos(img)
    print(len(cnts[0]))
    
        
    mask = np.zeros(img.shape, np.uint8)
    img_mask_cnts = cv.drawContours(mask, cnts, -1, (255,255,255),-1)
    img_mask_cnts=np.logical_not(img_mask_cnts)
    
    img[img_mask_cnts] = 0
    
    return img

In [5]:
#data="muestras" #fondo negro y sin rotar
#data="muestras_rotadas" fondo negro y con rotación
data="muestras_DA" #Data Augmentation Manual

# Preparando el datafrema  de entrenamiento (training)

In [6]:
# Dataframe Primera clase
count = 0
df_primera_e = pd.DataFrame()

#for imagen in glob('muestras_rotadas/Primera/Training/*.jpg'):
for imagen in glob(data + '/Primera/Training/*.jpg'):
    img = cv.imread(imagen)

    
    cnts= contornos(img)
    minRect = [None]*len(cnts)
    minEllipse = [None]*len(cnts)
    for i, c in enumerate(cnts):
        minRect[i] = cv.minAreaRect(c)
        if c.shape[0] > 5:
            minEllipse[i] = cv.fitEllipse(c)

    centerx_elipse= np.array(minEllipse[0][0])[0]
    centery_elipse= np.array(minEllipse[0][0])[1]

    large_elipse= np.array(minEllipse[0][1])[1]
    width_elipse= np.array(minEllipse[0][1])[0]

    area= area_elipse(large_elipse, width_elipse)
    perimetro=perimetro_elipse(large_elipse, width_elipse)


    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img_r = cv.resize(img_gray, (48,31), interpolation = cv.INTER_NEAREST)
    #img_r = cv.resize(img_gray, (95,62), interpolation = cv.INTER_NEAREST)
    datos= [1, centerx_elipse, centery_elipse, large_elipse, width_elipse, area, perimetro]
    fila = np.append(datos,  img_r.flatten())
    fila = fila.reshape(1, 1495  )
    #fila = fila.reshape(1, 5897  )
    df_primera_e = df_primera_e.append(pd.DataFrame(fila))
    count = count + 1
print(count)

2936


In [7]:
# Dataframe segunda clase
count = 0
df_segunda_e = pd.DataFrame()
for imagen in glob(data + '/Segunda/Training/*.jpg'):
    img = cv.imread(imagen)

    
    cnts= contornos(img)
    minRect = [None]*len(cnts)
    minEllipse = [None]*len(cnts)
    for i, c in enumerate(cnts):
        minRect[i] = cv.minAreaRect(c)
        if c.shape[0] > 5:
            minEllipse[i] = cv.fitEllipse(c)

    centerx_elipse= np.array(minEllipse[0][0])[0]
    centery_elipse= np.array(minEllipse[0][0])[1]

    large_elipse= np.array(minEllipse[0][1])[1]
    width_elipse= np.array(minEllipse[0][1])[0]

    area= area_elipse(large_elipse, width_elipse)
    perimetro=perimetro_elipse(large_elipse, width_elipse)


    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img_r = cv.resize(img_gray, (48,31), interpolation = cv.INTER_NEAREST)
    #img_r = cv.resize(img_gray, (95,62), interpolation = cv.INTER_NEAREST)
    datos= [2, centerx_elipse, centery_elipse, large_elipse, width_elipse, area, perimetro]
    fila = np.append(datos,  img_r.flatten())
    fila = fila.reshape(1, 1495  )
    #fila = fila.reshape(1, 5897  )
    df_segunda_e = df_segunda_e.append(pd.DataFrame(fila))
    count = count + 1
print(count)

4336


In [8]:
# Dataframe tercera clase
count = 0
df_tercera_e = pd.DataFrame()
for imagen in glob(data + '/Tercera/Training/*.jpg'):
    img = cv.imread(imagen)

    
    cnts= contornos(img)
    minRect = [None]*len(cnts)
    minEllipse = [None]*len(cnts)
    for i, c in enumerate(cnts):
        minRect[i] = cv.minAreaRect(c)
        if c.shape[0] > 5:
            minEllipse[i] = cv.fitEllipse(c)

    centerx_elipse= np.array(minEllipse[0][0])[0]
    centery_elipse= np.array(minEllipse[0][0])[1]

    large_elipse= np.array(minEllipse[0][1])[1]
    width_elipse= np.array(minEllipse[0][1])[0]

    area= area_elipse(large_elipse, width_elipse)
    perimetro=perimetro_elipse(large_elipse, width_elipse)


    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img_r = cv.resize(img_gray, (48,31), interpolation = cv.INTER_NEAREST)
    #img_r = cv.resize(img_gray, (95,62), interpolation = cv.INTER_NEAREST)
    datos= [3, centerx_elipse, centery_elipse, large_elipse, width_elipse, area, perimetro]
    fila = np.append(datos,  img_r.flatten())
    fila = fila.reshape(1, 1495  )
    #fila = fila.reshape(1, 5897  )
    df_tercera_e = df_tercera_e.append(pd.DataFrame(fila))
    count = count + 1
print(count)

2084


In [9]:
# Uniendo los dataframe y guardandolo en un csv
frames_e = [df_primera_e, df_segunda_e, df_tercera_e]
df_chiles_train = pd.concat(frames_e)
#df_chiles_train.to_csv('datos/df_train_sinrotar_3cat.csv', header=False, index=False)
df_chiles_train.to_csv('datos/df_train_DA_3cat.csv', header=False, index=False)

# Preparando el dataframe de prueba (test)

In [10]:
# Dataframe Primera clase
count = 0
df_primera_t = pd.DataFrame()
for imagen in glob(data + '/Primera/Test/*.jpg'):
    img = cv.imread(imagen)

    
    cnts= contornos(img)
    minRect = [None]*len(cnts)
    minEllipse = [None]*len(cnts)
    for i, c in enumerate(cnts):
        minRect[i] = cv.minAreaRect(c)
        if c.shape[0] > 5:
            minEllipse[i] = cv.fitEllipse(c)

    centerx_elipse= np.array(minEllipse[0][0])[0]
    centery_elipse= np.array(minEllipse[0][0])[1]

    large_elipse= np.array(minEllipse[0][1])[1]
    width_elipse= np.array(minEllipse[0][1])[0]

    area= area_elipse(large_elipse, width_elipse)
    perimetro=perimetro_elipse(large_elipse, width_elipse)


    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img_r = cv.resize(img_gray, (48,31), interpolation = cv.INTER_NEAREST)
    #img_r = cv.resize(img_gray, (95,62), interpolation = cv.INTER_NEAREST)
    datos= [1, centerx_elipse, centery_elipse, large_elipse, width_elipse, area, perimetro]
    fila = np.append(datos,  img_r.flatten())
    fila = fila.reshape(1, 1495  )
    #fila = fila.reshape(1, 5897  )
    df_primera_t = df_primera_t.append(pd.DataFrame(fila))
    count = count + 1
print(count)

320


In [11]:
# Dataframe segunda clase
count = 0
df_segunda_t = pd.DataFrame()
for imagen in glob(data + '/Segunda/Test/*.jpg'):
    img = cv.imread(imagen)

    
    cnts= contornos(img)
    minRect = [None]*len(cnts)
    minEllipse = [None]*len(cnts)
    for i, c in enumerate(cnts):
        minRect[i] = cv.minAreaRect(c)
        if c.shape[0] > 5:
            minEllipse[i] = cv.fitEllipse(c)

    centerx_elipse= np.array(minEllipse[0][0])[0]
    centery_elipse= np.array(minEllipse[0][0])[1]

    large_elipse= np.array(minEllipse[0][1])[1]
    width_elipse= np.array(minEllipse[0][1])[0]

    area= area_elipse(large_elipse, width_elipse)
    perimetro=perimetro_elipse(large_elipse, width_elipse)


    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img_r = cv.resize(img_gray, (48,31), interpolation = cv.INTER_NEAREST)
    #img_r = cv.resize(img_gray, (95,62), interpolation = cv.INTER_NEAREST)
    datos= [2, centerx_elipse, centery_elipse, large_elipse, width_elipse, area, perimetro]
    fila = np.append(datos,  img_r.flatten())
    fila = fila.reshape(1, 1495  )
    #fila = fila.reshape(1, 5897  )
    df_segunda_t = df_segunda_t.append(pd.DataFrame(fila))
    count = count + 1
print(count)

480


In [12]:
# Dataframe tercera clase
count = 0
df_tercera_t = pd.DataFrame()
for imagen in glob(data + '/Tercera/Test/*.jpg'):
    img = cv.imread(imagen)

    
    cnts= contornos(img)
    minRect = [None]*len(cnts)
    minEllipse = [None]*len(cnts)
    for i, c in enumerate(cnts):
        minRect[i] = cv.minAreaRect(c)
        if c.shape[0] > 5:
            minEllipse[i] = cv.fitEllipse(c)

    centerx_elipse= np.array(minEllipse[0][0])[0]
    centery_elipse= np.array(minEllipse[0][0])[1]

    large_elipse= np.array(minEllipse[0][1])[1]
    width_elipse= np.array(minEllipse[0][1])[0]

    area= area_elipse(large_elipse, width_elipse)
    perimetro=perimetro_elipse(large_elipse, width_elipse)


    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img_r = cv.resize(img_gray, (48,31), interpolation = cv.INTER_NEAREST)
    #img_r = cv.resize(img_gray, (95,62), interpolation = cv.INTER_NEAREST)
    datos= [3, centerx_elipse, centery_elipse, large_elipse, width_elipse, area, perimetro]
    fila = np.append(datos,  img_r.flatten())
    fila = fila.reshape(1, 1495  )
    #fila = fila.reshape(1, 5897  )
    df_tercera_t = df_tercera_t.append(pd.DataFrame(fila))
    count = count + 1
print(count)

232


In [13]:
# Uniendo los dataframe y guardandolo en un csv
frames_e = [df_primera_t, df_segunda_t, df_tercera_t]
df_chiles_test = pd.concat(frames_e)
#df_chiles_test.to_csv('datos/df_test_sinrotar_3cat.csv', header=False, index=False)
df_chiles_test.to_csv('datos/df_test_DA_3cat.csv', header=False, index=False)